<h1><span style = "color: rgb(51,153,102)">Import libraries</span></h1>

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

<h1><span style = "color: rgb(51,153,102)">Read data into a data frame</span></h1>

In [63]:
!ls -l

total 964
-rwxrwxr-x 1 quang quang 963738 Jan 22 14:01 AmesHousing.tsv
-rwxrwxr-x 1 quang quang  19254 Jan 22 19:05 Predicting_House_Sale_Prices.ipynb


In [64]:
housing = pd.read_csv('AmesHousing.tsv', sep='\t')
housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [65]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              198 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         29

<h1><span style = "color: rgb(51,153,102)"> Create <span style="background-color: rgba(51,153,102, 0.24)">transform_features()</span></span></h1>

This function performs:
- Drop <span style="background-color: rgba(153,0,255, 0.2)">Mo Sold</span> and <span style="background-color: rgba(153,0,255, 0.2)">Yr Sold</span> columns since that leaks information about <span style="background-color: rgba(153,0,255, 0.2)">SalePrice</span>.
- Drop columns <span style="background-color: rgba(153,0,255, 0.2)">Order</span> and <span style="background-color: rgba(153,0,255, 0.2)">PID</span> that aren't useful for machine learning.
- Drop columns that has more than 5% missing values.
- Remove any houses with more than 4000 square feet from the data set, which are outliers.
- For numerical columns that has less than 5% missing values, let's fill in the missing values using the most popular value for that column.
- Create feature <span style="background-color: rgba(153,0,255, 0.2)">Mo Sold</span> and <span style="background-color: rgba(153,0,255, 0.2)">years_until_remod</span> and drop columns <span style="background-color: rgba(153,0,255, 0.2)">Year Remod/Add</span>, <span style="background-color: rgba(153,0,255, 0.2)">Year Built</span>.

In [104]:
def transform_features(data):
    new_df = data.drop(['Mo Sold', 'Yr Sold', 'Order', 'PID'], axis=1)
    
    null_counts = new_df.isnull().sum()    
    new_df = new_df[null_counts[((null_counts / data.shape[0]) < 0.25)].index]
    
    new_df = new_df[new_df['Gr Liv Area'] < 4000]
    
    num_cols = new_df.select_dtypes(['int64', 'float64']).columns
    
    for col in num_cols:
        if new_df[col].isnull().sum() > 0:
            fill_value = new_df[col].value_counts().index[0]
            new_df = new_df.fillna(fill_value)
            
    new_df['years_until_remod'] = new_df['Year Remod/Add'] - new_df['Year Built']
    new_df = new_df.drop(['Year Remod/Add', 'Year Built'], axis=1)
    
    return new_df

In [105]:
new = transform_features(housing)
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2925 entries, 0 to 2929
Data columns (total 72 columns):
MS SubClass          2925 non-null int64
MS Zoning            2925 non-null object
Lot Frontage         2925 non-null float64
Lot Area             2925 non-null int64
Street               2925 non-null object
Lot Shape            2925 non-null object
Land Contour         2925 non-null object
Utilities            2925 non-null object
Lot Config           2925 non-null object
Land Slope           2925 non-null object
Neighborhood         2925 non-null object
Condition 1          2925 non-null object
Condition 2          2925 non-null object
Bldg Type            2925 non-null object
House Style          2925 non-null object
Overall Qual         2925 non-null int64
Overall Cond         2925 non-null int64
Roof Style           2925 non-null object
Roof Matl            2925 non-null object
Exterior 1st         2925 non-null object
Exterior 2nd         2925 non-null object
Mas Vnr Type  

<h1><span style = "color: rgb(51,153,102)"> Create <span style="background-color: rgba(51,153,102, 0.24)">select_features()</span></span></h1>

In [106]:
def select_features(data):
    return data[['Gr Liv Area', 'SalePrice']]

<h1><span style = "color: rgb(51,153,102)"> Create <span style="background-color: rgba(51,153,102, 0.24)">train_and_test()</span></span></h1>

The <span style="background-color: rgba(153,0,255, 0.2)">train_and_test()</span> function performs:
- Selects the first <span style="background-color: rgba(153,0,255, 0.2)">1460</span> rows from from data and assign to <span style="background-color: rgba(153,0,255, 0.2)">train</span>.
- Selects the remaining rows from data and assign to <span style="background-color: rgba(153,0,255, 0.2)">test</span>.
- Trains a model using all numerical columns except the <span style="background-color: rgba(153,0,255, 0.2)">SalePrice</span> column (the target column) from the data frame returned from <span style="background-color: rgba(153,0,255, 0.2)">select_features()</span>.
- Tests the model on the test set using and returns the RMSE value.

In [107]:
def train_and_test(data):
    train = data[:1460]
    test = data[1460:]
    num_cols = train.select_dtypes(['int64', 'float64']).columns
    target = 'SalePrice'
    features = num_cols.drop(target)
    
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    prediction = lr.predict(test[features])
    rmse = np.sqrt(mean_squared_error(prediction, test[target]))
    
    return rmse

In [108]:
housing_tf = transform_features(housing)
housing_sf = select_features(housing_tf)
rmse = train_and_test(housing_sf)
rmse

53033.912197397425